In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# probability model
from scipy.stats import multivariate_normal

In [29]:
# Use the I3D-based boundaries you generated (var-T, FNS ✗)
nalist = np.load(r"..\list\nalist_i3d.npy")
total_T = int(nalist[-1, 1])

# concat_UCF.npy was created via np.memmap (raw float32 file), so load with memmap
train_data = np.memmap(r"..\concat_UCF.npy", dtype="float32", mode="r", shape=(total_T, 10, 2048))

print("nalist shape:", nalist.shape, " total_T:", total_T)
print("train_data shape:", train_data.shape)

nalist shape: (1609, 2)  total_T: 779951
train_data shape: (779951, 10, 2048)


In [30]:
# Sanity check: video-0 span in the concatenated memmap
a0, b0 = map(int, nalist[0])
print("video0 span:", (a0, b0), "T0:", b0-a0)

video0 span: (0, 171) T0: 171


In [31]:
new_repr = []
for i, (fromid, toid) in enumerate(nalist):
    new_repr.append(train_data[fromid:toid])

len(new_repr)

1609

In [34]:
nalist_i3d = np.load(r"..\list\nalist_i3d.npy")
from0, to0 = nalist_i3d[0]
print("nalist[0]:", from0, to0, "len=", to0-from0)

x0 = train_data[from0:to0]
print("slice x0:", x0.shape)        # 여기서 (171,10,2048) 나와야 정상

print("new_repr[0]:", new_repr[0].shape)


nalist[0]: 0 171 len= 171
slice x0: (171, 10, 2048)
new_repr[0]: (171, 10, 2048)


In [35]:
new_repr[i].shape

(151, 10, 2048)

In [36]:
def estimate_gauss(X):
    m = X.shape[0]   # using only first dimension as we know it has only one feature - l2 norm
    
    mu = np.mean(X, axis=0)
    var = np.cov(X.T)
    
    return mu, var

def covariance_mat(X):
    X = np.mean(X , axis= 1)
    X =  X.transpose(1,0)
    cov  = np.cov(X)

    return cov

def get_matrix(data):

    l2_norm = np.sum(np.square(data), axis=2)
    n_train_crop_l2_norm_mean = np.mean(l2_norm, axis= 1)

    return n_train_crop_l2_norm_mean


def diff_l2(new_repr):

    l2_norms = []
    for i in range(len(new_repr)):
        l2_norms.append(get_matrix(new_repr[i]))

    mean_v_l2 = []
    for i in range(len(l2_norms)):
        mean_v_l2.append(np.diff(l2_norms[i], n=1).max())
    return mean_v_l2


In [37]:
params = []
for i in range(len(new_repr)):

    param = get_matrix(new_repr[i])
    mu, var = estimate_gauss(param)

    params.append((mu, var, ))

In [39]:
np.array(params).shape

(1609, 2)

In [40]:
from sklearn.mixture import GaussianMixture
import time


gmm = GaussianMixture(n_components=2, max_iter=150, random_state=0, covariance_type='spherical')
# gmm_scores = gmm.score_samples(params)
labels = gmm.fit_predict(params)

y_gmm = gmm.fit_predict(params)
print(y_gmm.sum(), y_gmm.sum() / len(y_gmm))


score = y_gmm 
score = gmm.score_samples(params) 
pct_threshold = np.percentile(score, 3)
print(f'The threshold of the score is {pct_threshold:.2f}') 
res = np.array([1 if x < pct_threshold else 0 for x in score]) 
print(res.sum())

427 0.2653822249844624
The threshold of the score is -23.32
49


In [41]:
abnormal_portion = np.where(labels == 1)[0]
normal_portion = np.where(labels == 0)[0]
normal_portion.shape, abnormal_portion.shape

((1182,), (427,))

In [42]:
n_params = np.array(params)[normal_portion]
a_params = np.array(params)[abnormal_portion]
n_params.shape, a_params.shape

((1182, 2), (427, 2))

In [43]:
abag = list(zip(list(np.array(params)[abnormal_portion]), abnormal_portion))
nbag = list(zip(list(np.array(params)[normal_portion]), normal_portion))
len(abag), len(nbag)

(427, 1182)

In [51]:
print("len(nbag):", len(nbag))
print("type(nbag[0]):", type(nbag[0]))

# nbag[0]이 튜플/리스트면 feature가 보통 [0]에 있음
try:
    x0 = nbag[0][0]
    print("type(nbag[0][0]):", type(x0))
    print("shape(nbag[0][0]):", np.array(x0).shape)
except Exception as e:
    print("nbag[0][0] 접근 실패:", e)
    print("shape(nbag[0]):", np.array(nbag[0]).shape)


len(nbag): 1182
type(nbag[0]): <class 'tuple'>
type(nbag[0][0]): <class 'numpy.ndarray'>
shape(nbag[0][0]): (2,)


In [45]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
from sklearn.mixture import GaussianMixture

In [ ]:

nu = 1.0
step = 1
import time
start = time.time()
while len(abag) / len(nbag) < nu:
    
    temp_bag = nbag
    X = np.vstack([np.asarray(t[0], dtype=np.float32).reshape(-1) for t in temp_bag])  # (N, 2)
    y_gmm = gmm.fit_predict(X)
    score = gmm.score_samples(X)

    pct_threshold = np.percentile(score, 3) 
    res = np.array([1 if x < pct_threshold else 0 for x in score]) 
    print(f'The threshold of the score in step {step} is {pct_threshold:.2f}, abnormal part: {res.sum()}') 
    
    abnormal_portion = np.where(res == 1)[0]
    normal_portion = np.where(res == 0)[0]
    # abnormal_portion / normal_portion 은 index 배열 (np.where 결과)
    abag += [temp_bag[i] for i in abnormal_portion.tolist()]
    nbag  = [temp_bag[i] for i in normal_portion.tolist()]
    step += 1

print(time.time() - start)

The threshold of the score in step 1 is -19.88, abnormal part: 36
The threshold of the score in step 2 is -19.53, abnormal part: 35
The threshold of the score in step 3 is -19.18, abnormal part: 34
The threshold of the score in step 4 is -19.05, abnormal part: 33
The threshold of the score in step 5 is -18.95, abnormal part: 31
The threshold of the score in step 6 is -18.98, abnormal part: 31
The threshold of the score in step 7 is -18.87, abnormal part: 30
The threshold of the score in step 8 is -18.74, abnormal part: 29
The threshold of the score in step 9 is -18.47, abnormal part: 28
The threshold of the score in step 10 is -18.30, abnormal part: 27
The threshold of the score in step 11 is -18.24, abnormal part: 27
The threshold of the score in step 12 is -18.13, abnormal part: 26
The threshold of the score in step 13 is -17.96, abnormal part: 25
0.14249253273010254


In [54]:
print(np.where(np.array([x[1] for x in abag]) < 810)[0].shape, len([x[1] for x in abag]))
print('correctness acc: ', np.where(np.array([x[1] for x in abag]) < 810)[0].shape[0] / len([x[1] for x in abag]))

(442,) 819
correctness acc:  0.5396825396825397


In [55]:
print(np.where(np.array([x[1] for x in nbag]) > 810)[0].shape, len([x[1] for x in nbag]))
print('correctness acc: ', np.where(np.array([x[1] for x in nbag]) > 810)[0].shape[0] / len([x[1] for x in nbag]))

(422,) 790
correctness acc:  0.5341772151898734


In [56]:
temp = [k[1] for k in sorted([(x[1], 1.0) for x in abag] + [(x[1], 0.0) for x in nbag], key=lambda z: z[0])]
sum(temp), len(temp)

(819.0, 1609)

In [57]:
# normal set creation
normal_set = {}

for i in range(len(new_repr)):
    if temp[i] == 0.0:
        normal_set[i] = new_repr[i]

In [58]:
# abnormal set creation
abnormal_set = {}
for i in range(len(new_repr)):
    if temp[i] == 1.0:
        abnormal_set[i] = new_repr[i]

In [59]:
l2_norms_N = np.empty(0,)
for (idel, sample) in normal_set.items():
    
    # print(sample.shape)
    

    l2_norms_N = np.append(l2_norms_N,get_matrix(sample))

In [60]:
len(normal_set), len(abnormal_set)

(790, 819)

In [61]:
mu_GMM, var_GMM = estimate_gauss(np.array(l2_norms_N))

In [62]:
# probability model
from scipy.stats import multivariate_normal
p = multivariate_normal(mu_GMM, var_GMM)

In [63]:
ground_truth = {} 
length = 0.2 
for (idel, sample) in abnormal_set.items(): 

    # feature extraction 
    # sample_matrix = np.sum(np.square(sample), axis=1)  # for just l2
    sample_matrix = get_matrix(sample)
    
    # get p values
    probs = p.pdf(sample_matrix)
    temp_list = []
    temp_list += [0.0] * len(probs)
    
    window_size = int(len(probs) * length)  # fixed
    temp = []
    for idx in range(0, len(probs) - window_size + 1):
        arr = 0
        for i in range(idx, idx + window_size - 1):
            arr += abs(probs[i+1] - probs[i])
        temp.append(arr)

    for i in range(temp.index(max(temp)), temp.index(max(temp)) + window_size):
        temp_list[i] = 1.0

    ground_truth[idel] = temp_list

In [64]:
final_gt = []
abnormal_gt = []
for i in range(len(new_repr)):
    if i in normal_set.keys():
        final_gt += [0.0] * new_repr[i].shape[0]
    else:
        final_gt += ground_truth[i]
        abnormal_gt+= ground_truth[i]

In [65]:
len(final_gt)

779951

In [67]:
final_gt = np.array(final_gt, dtype=np.float32)

print("final_gt shape:", final_gt.shape, "sum:", final_gt.sum())
np.save(r"..\Unsup_labels\UCF_unsup_labels_i3d_varT.npy", final_gt)

final_gt shape: (779951,) sum: 57267.0


In [68]:
print("final abag:", len(abag), "nbag:", len(nbag), "ratio:", len(abag)/len(nbag))


final abag: 819 nbag: 790 ratio: 1.0367088607594936


In [ ]:
pseudo = np.load("../Unsup_labels/UCF_unsup_labels_i3d_varT.npy")
print(pseudo.shape, pseudo.dtype)
print("mean:", pseudo.mean(), "sum:", pseudo.sum()) #sum 결과 = 전체 스니펫(779951개) 중 약 5.7만 개를 이상(1)으로 찍었음을 알 수 있음. 
print("ones ratio:", pseudo.mean())


(779951,) float32
mean: 0.07342384 sum: 57267.0
ones ratio: 0.07342384


In [ ]:
np.save('unsupervised_PL/'+'UCF_labels_entropy_1.npy', final_gt) #UCF
# np.save('Unsup_labels/'+'XD_I3D_unsup_labels_10_V2_GMM.npy', final_gt) #XD